In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.max_rows = 100

import numpy as np

# import matplotlib.pyplot as plt
# plt.rcParams.update({'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})

# import seaborn as sns
import pickle
import joblib

# %matplotlib inline

from ipywidgets import interact

In [2]:
Farmer_data = pd.read_csv("Farmer2_Data.csv")
Farmer_data.head(10)

,Farmer,Supply Capacity,Payment Terms,Technical Support,Quality Terms,Financial Stability,Unit Cost,On Time Delivery,unit_price,Defective_rate,on_time_delivery_rate,supply_capacity,Demand
0,1,0.42,0.42,0.96,0.49,0.94,0.13,0.78,255,0.06,0.99,200,394
1,2,0.82,0.43,0.55,0.65,0.76,0.74,0.21,284,0.08,0.98,300,655
2,3,0.26,0.55,0.44,0.77,0.52,0.87,0.14,199,0.02,0.89,300,749
3,4,0.75,0.66,0.14,0.81,0.16,0.44,0.71,114,0.06,0.91,300,238
4,5,0.90,0.14,0.88,0.89,0.94,0.66,0.22,283,0.01,0.98,300,499
5,6,0.34,0.11,0.60,0.92,0.45,0.50,0.61,262,0.05,0.96,300,935
6,7,0.48,0.74,0.64,0.55,0.28,0.97,0.31,273,0.08,0.98,500,865
7,8,0.15,0.69,0.45,0.95,0.45,0.31,0.25,112,0.02,0.87,500,108
8,9,0.21,0.44,0.24,0.47,0.29,0.22,0.95,139,0.04,0.96,500,416
9,10,0.30,1.00,0.21,0.62,0.26,0.19,0.36,91,0.02,0.91,700,925


In [3]:
# Scores for our hypothetical supplier 
# Required_Qty= 4
Payment_Terms = 0.23
Technical_Support = 0.63
Quality_Terms= 0.97
Financial_Stability = 0.10

# Weights for each subjective factor 
# weight_Required_Qty = 0.30
weight_Payment_Terms = 0.60
weight_Technical_Support = 0.40
weight_Quality_Terms = 0.70
weight_Financial_Stability = 0.90

# Computations
sum_of_weights = weight_Payment_Terms +\
weight_Technical_Support + weight_Quality_Terms + weight_Financial_Stability

# Normalization step for weights
# weight_Required_Quantity = weight_Required_Quantity/sum_of_weights
weight_Payment_Terms = weight_Payment_Terms/sum_of_weights
weight_Technical_Support = weight_Technical_Support/sum_of_weights
weight_Quality_Terms = weight_Quality_Terms/sum_of_weights
weight_Financial_Stability = weight_Financial_Stability/sum_of_weights

# Use normalized weights to calculate supplier's score
supplier_score =  weight_Payment_Terms*Payment_Terms + \
weight_Technical_Support*Technical_Support + weight_Quality_Terms*Quality_Terms +\
weight_Financial_Stability*Financial_Stability

print("The farmer's weighted score is "+str(supplier_score))

The farmer's weighted score is 0.44576923076923075


In [4]:
def Compute_Weighted_Score_Sum(df,criteria_list,weights_array):
    normalized_weights = weights_array/weights_array.sum()
    
    df['Weighted_Score_Sum'] = 0
    for i in range(len(criteria_list)):
        current_criteria = criteria_list[i]
        current_weight = normalized_weights[i]
        df['Weighted_Score_Sum'] += current_weight*df[current_criteria]

    return df

In [5]:
criteria = [
            'Payment Terms',
            'Technical Support',
            'Quality Terms',
            'Financial Stability'
           ]

criteria_weights = np.array([6,4,7,9])

if len(criteria) != len(criteria_weights):
    print('The number of criteria and weights that you specified do not match!')
else:
    Farmer_data = Compute_Weighted_Score_Sum(Farmer_data,criteria,criteria_weights)

In [6]:
Farmer_data.head()

,Farmer,Supply Capacity,Payment Terms,Technical Support,Quality Terms,Financial Stability,Unit Cost,On Time Delivery,unit_price,Defective_rate,on_time_delivery_rate,supply_capacity,Demand,Weighted_Score_Sum
0,1,0.42,0.42,0.96,0.49,0.94,0.13,0.78,255,0.06,0.99,200,394,0.70
1,2,0.82,0.43,0.55,0.65,0.76,0.74,0.21,284,0.08,0.98,300,655,0.62
2,3,0.26,0.55,0.44,0.77,0.52,0.87,0.14,199,0.02,0.89,300,749,0.58
3,4,0.75,0.66,0.14,0.81,0.16,0.44,0.71,114,0.06,0.91,300,238,0.45
4,5,0.90,0.14,0.88,0.89,0.94,0.66,0.22,283,0.01,0.98,300,499,0.73


In [7]:
from collections import Counter
np.random.seed(42)

criteria = [
            'Payment Terms',
            'Technical Support',
            'Quality Terms',
            'Financial Stability']

criteria_weights = np.array([6,4,7,9])

perturbations = 1000
min_perturbation = -2.0
max_perturbation = 2.0
Top_Farmers = 15

farmer_list = []
if len(criteria) != len(criteria_weights):
    print('The number of criteria and weights that you specified do not match!')
else:
    for _ in range(perturbations):
        perturbed_weights = np.random.uniform(low=min_perturbation,
                                              high=max_perturbation+0.1,
                                              size=len(criteria_weights)) + criteria_weights
        perturbed_weights = np.maximum(perturbed_weights,0)
        perturbed_weights = np.minimum(perturbed_weights,10)
        data = Compute_Weighted_Score_Sum(Farmer_data, criteria, perturbed_weights)
        temp = data.nlargest(Top_Farmers,columns = 'Weighted_Score_Sum')
        top_farmers_list = temp['Farmer'].values.tolist()
        farmer_list.append(top_farmers_list)

sum_farmer_counts = Counter(x for sublist in farmer_list for x in sublist)
sum_farmer_counts = pd.DataFrame.from_dict(sum_farmer_counts, orient='index').reset_index()
sum_farmer_counts = sum_farmer_counts.rename(columns={'index':'Farmer', 0:'Count'})
sum_farmer_counts.sort_values(by='Count',inplace=True,ascending=False)
sum_farmer_counts['Weighted_Sum_Proportion'] = sum_farmer_counts['Count']/perturbations
sum_farmer_counts.set_index('Farmer')

,Count,Weighted_Sum_Proportion
Farmer,,
27,1000,1.00
71,1000,1.00
54,1000,1.00
53,1000,1.00
40,1000,1.00
20,1000,1.00
67,1000,1.00
91,1000,1.00
38,1000,1.00


In [8]:
merged_data=sum_farmer_counts
h = merged_data.merge(Farmer_data, on= 'Farmer', how= 'outer') 

# h.drop(labels = ['Count'], axis =1, inplace=True)

h

,Farmer,Count,Weighted_Sum_Proportion,Supply Capacity,Payment Terms,Technical Support,Quality Terms,Financial Stability,Unit Cost,On Time Delivery,unit_price,Defective_rate,on_time_delivery_rate,supply_capacity,Demand,Weighted_Score_Sum
0,27,1000.00,1.00,0.55,0.85,0.84,0.65,0.95,0.19,0.28,266,0.08,0.99,600,112,0.82
1,71,1000.00,1.00,0.81,0.78,0.76,0.70,0.97,0.23,0.63,170,0.09,0.92,200,986,0.81
2,54,1000.00,1.00,0.26,0.74,0.84,0.84,0.79,0.80,0.60,70,0.02,0.92,600,105,0.80
3,53,1000.00,1.00,0.24,0.80,0.63,0.71,0.98,0.35,0.73,114,0.06,0.98,600,648,0.80
4,40,1000.00,1.00,0.62,0.97,0.60,0.93,0.68,0.98,0.77,267,0.04,1.00,300,815,0.81
5,20,1000.00,1.00,0.99,0.95,0.59,0.77,0.82,0.83,0.86,153,0.04,0.90,700,375,0.80
6,67,1000.00,1.00,0.28,0.64,0.98,0.52,0.93,0.44,0.65,243,0.06,0.88,300,209,0.75
7,91,1000.00,1.00,0.21,0.76,0.90,0.64,0.80,0.55,0.20,252,0.03,0.94,500,852,0.76
8,38,1000.00,1.00,0.37,0.61,0.67,0.81,0.81,0.38,0.50,297,0.08,0.94,400,124,0.74
9,31,963.00,0.96,0.42,0.65,0.47,0.83,0.86,0.54,0.79,76,0.01,0.92,400,979,0.74


In [9]:
from gurobipy import*
model = Model("Suppliers")
import pandas as pd


--------------------------------------------
--------------------------------------------

Using license file C:\Users\prasangika\gurobi.lic
Academic license - for non-commercial use only


In [10]:
I = {1}
df = h
R = {0,1,2,3}
J=h.iloc[0:8, 0:1]
J


,Farmer
0,27
1,71
2,54
3,53
4,40
5,20
6,67
7,91


In [11]:
J,w = multidict({0:df.at[0,'Weighted_Score_Sum'], 1:df.at[1,'Weighted_Score_Sum'], 2:df.at[2,'Weighted_Score_Sum'], 3:df.at[3,'Weighted_Score_Sum'],4:df.at[4,'Weighted_Score_Sum'],5:df.at[5,'Weighted_Score_Sum'],6:df.at[6,'Weighted_Score_Sum'],7:df.at[7,'Weighted_Score_Sum']})
J
w

{0: 0.8204250691509556,
 1: 0.8139807958731873,
 2: 0.8013743505321651,
 3: 0.8049751653631303,
 4: 0.8085040761827671,
 5: 0.799327519467755,
 6: 0.7487668256905431,
 7: 0.7579943036427277}

In [12]:
J,q = multidict({0:df.at[0,'Defective_rate'], 1:df.at[1,'Defective_rate'], 2:df.at[2,'Defective_rate'], 3:df.at[3,'Defective_rate'], 4:df.at[4,'Defective_rate'],5:df.at[5,'Defective_rate'],6:df.at[6,'Defective_rate'],7:df.at[7,'Defective_rate']})
J
q

{0: 0.08, 1: 0.09, 2: 0.02, 3: 0.06, 4: 0.04, 5: 0.04, 6: 0.06, 7: 0.03}

In [13]:
J,t = multidict({0:df.at[0,'on_time_delivery_rate'], 1:df.at[1,'on_time_delivery_rate'], 2:df.at[2,'on_time_delivery_rate'], 3:df.at[3,'on_time_delivery_rate'],4:df.at[4,'on_time_delivery_rate'],5:df.at[5,'on_time_delivery_rate'],6:df.at[6,'on_time_delivery_rate'],7:df.at[7,'on_time_delivery_rate']})
J,t

([0, 1, 2, 3, 4, 5, 6, 7],
 {0: 0.99, 1: 0.92, 2: 0.92, 3: 0.98, 4: 1.0, 5: 0.9, 6: 0.88, 7: 0.94})

In [14]:
J,C =  multidict({0:df.at[0,'supply_capacity'], 1:df.at[1,'supply_capacity'], 2:df.at[2,'supply_capacity'], 3:df.at[3,'supply_capacity'],4:df.at[4,'supply_capacity'],5:df.at[5,'supply_capacity'],6:df.at[6,'supply_capacity'],7:df.at[7,'supply_capacity']})
C

{0: 600, 1: 200, 2: 600, 3: 600, 4: 300, 5: 700, 6: 300, 7: 500}

In [15]:
J,p = multidict({0:df.at[0,'unit_price'], 1:df.at[1,'unit_price'], 2:df.at[2,'unit_price'], 3:df.at[3,'unit_price'],4:df.at[4,'unit_price'],5:df.at[5,'unit_price'],6:df.at[6,'unit_price'],7:df.at[7,'unit_price']})
p

{0: 266, 1: 170, 2: 70, 3: 114, 4: 267, 5: 153, 6: 243, 7: 252}

In [16]:
I,Demand = multidict({1:400})

In [17]:
I,defective_rate = multidict({1:0.03})

In [18]:
I,on_time = multidict({1:0.9})

In [19]:
d = {(0,1):0, (0,2):0.1, (0,3):0.2,
    (1,1):0, (1,2):0.1, (1,3):0.2,
    (2,1):0, (2,2):0.1, (2,3):0.2,
    (3,1):0, (3,2):0.1, (3,3):0.2,
    (4,1):0, (4,2):0.1, (4,3):0.2,
    (5,1):0, (5,2):0.1, (5,3):0.2,
    (6,1):0, (6,2):0.1, (6,3):0.2,
    (7,1):0, (7,2):0.1, (7,3):0.2,
    }
d

{(0, 1): 0,
 (0, 2): 0.1,
 (0, 3): 0.2,
 (1, 1): 0,
 (1, 2): 0.1,
 (1, 3): 0.2,
 (2, 1): 0,
 (2, 2): 0.1,
 (2, 3): 0.2,
 (3, 1): 0,
 (3, 2): 0.1,
 (3, 3): 0.2,
 (4, 1): 0,
 (4, 2): 0.1,
 (4, 3): 0.2,
 (5, 1): 0,
 (5, 2): 0.1,
 (5, 3): 0.2,
 (6, 1): 0,
 (6, 2): 0.1,
 (6, 3): 0.2,
 (7, 1): 0,
 (7, 2): 0.1,
 (7, 3): 0.2}

In [20]:
b = {(0,0):0, (0,1):10000, (0,2):20000, (0,3):10000000,
    (1,0):0, (1,1):10000, (1,2):20000, (1,3):10000000,
    (2,0):0, (2,1):10000, (2,2):20000, (2,3):10000000,
    (3,0):0, (3,1):10000, (3,2):20000, (3,3):10000000,
    (4,0):0, (4,1):10000, (4,2):20000, (4,3):10000000,
    (5,0):0, (5,1):10000, (5,2):20000, (5,3):10000000,
    (6,0):0, (6,1):10000, (6,2):20000, (6,3):10000000,
    (7,0):0, (7,1):10000, (7,2):20000, (7,3):10000000,
    }

In [21]:
#Binary integer variable, equal to 1 if business volume purchased from supplier j falls on the discount interval r
y = {}
for j in J:
    for r in R:
        y[j,r] = model.addVar(vtype="b", name="y(%s,%s)"%(j,r))

In [22]:
#Units of item i to purchase from supplier j
x = {}
for i in I:
    for j in J:
        x[i,j] = model.addVar(lb=0, vtype="c", name="x(%s,%s)"%(i,j))

In [23]:
# Business volume purchased from supplier j in discount interval r
v = {}
for j in J:
    for r in R:
        v[j,r] = model.addVar(vtype="c", name="v(%s,%s)"%(j,r))

In [24]:
# Maximizing total weighted quantity of purchasing
model.setObjectiveN(-quicksum(w[j]*x[i,j] for i,j in x),0)

In [25]:
#Minimize the total purchase cost
model.setObjectiveN(quicksum((1-d[j,r])*v[j,r] for j in J for r in range(1,4)),1)

In [26]:
#Minimize the number of defective items
model.setObjectiveN(quicksum(q[j]*x[i,j] for i,j in x),2)

In [27]:
#Maximizing the number of items delivered on time
model.setObjectiveN(-quicksum(t[j]*x[i,j] for i,j in x),3)

In [28]:
model.ModelSense = GRB.MINIMIZE

In [29]:
model.addConstr(quicksum(v[j,r] for r in range(1,4) for j in J) == quicksum(p[j]*x[1,j] for j in J))

<gurobi.Constr *Awaiting Model Update*>

In [30]:
#Capacity constraint
for j in J:
    for i in I:
        model.addConstr(x[i,j] <= C[j])

In [31]:
#Discount constraint
for j in J:
    model.addConstr(quicksum(y[j,r] for r in range(1,4)) <= 1)

In [32]:
for j in J:
    for r in range(1,4):
        model.addConstr(b[j,r-1]*y[j,r] <= v[j,r])

In [33]:
for j in J:
    for r in range(1,4):
        model.addConstr(v[j,r] <= (b[j,r]-1)*y[j,r])

In [34]:
# Demand constraint
for i in I:
    model.addConstr(quicksum(x[i,j] for j in J) == Demand[i])

In [35]:
#Quality constraint
for i in I:
    model.addConstr(quicksum(x[i,j]*q[j] for j in J) <= Demand[i]*defective_rate[i])

In [36]:
#Delivery constraint
for i in I:
    model.addConstr(quicksum((1-t[j])*x[i,j] for j in J) <= (1-on_time[i])*Demand[i])

In [37]:
model.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 68 rows, 72 columns and 175 nonzeros
Model fingerprint: 0x325f4da7
Variable types: 40 continuous, 32 integer (32 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+07]
  Objective range  [2e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 4 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 68 rows, 72 columns and 175 nonzeros
Variable types: 40 continuous, 32 integer (32 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+07]
  Objective range  [8e-01, 2e+00]
 

In [38]:
assert model.Status == GRB.Status.OPTIMAL

In [39]:
nSolutions = model.SolCount
nObjectives = model.NumObj
print('Problem has', nObjectives, 'objectives')
print('Gurobi found', nSolutions, 'solutions')

Problem has 4 objectives
Gurobi found 1 solutions


In [40]:
solutions = []
#for s in range(nSolutions):
#    model.params.SolutionNumber = s
#    print('Solution', s, ':', end='')
for o in range(nObjectives):
    model.params.ObjNumber = o
    print('Objective value ',o+1, ': ', model.ObjNVal, end='')

Objective value  1 :  -320.549740212866Objective value  2 :  22400.0Objective value  3 :  8.0Objective value  4 :  -368.0

In [41]:
solutions.append(model.getAttr('Xn',x))
solutions


[{(1, 0): 0.0,
  (1, 1): 0.0,
  (1, 2): 400.0,
  (1, 3): 0.0,
  (1, 4): 0.0,
  (1, 5): 0.0,
  (1, 6): 0.0,
  (1, 7): 0.0}]

In [42]:
EPS = 0.00000001
for (i,j) in x:
    if x[i,j].X > EPS:
        print("Item %1s bought from supplier %1s is: %3s"%(i,j, x[i,j].X))

Item 1 bought from supplier 2 is: 400.0


In [43]:
# with open('Finalized_FarmerModel.sav', 'wb') as f:
#     pickle.dump(df, f)

filename='Finalized_FarmerModel.sav' 
joblib.dump(Model, filename)

# model.write("Finalized_FarmerModel.sol")

# model.write("Finalized_FarmerModel.mps")

['Finalized_FarmerModel.sav']

In [44]:
#load model from the disk
# with open('Finalized_FarmerModel.sav', 'rb') as f:
#     df = pickle.load(f)
# result= solutions

loaded_model = joblib.load('C:\Users\prasangika\Finalized_FarmerModel.sav')
result = loaded_model(Demand, defective_rate)
result

# read("Finalized_FarmerModel.mps") as f:
    

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-44-d70c0ea4d038>, line 6)